In [1]:
import log_reg
from imp import reload

import numpy as np
import pandas as pd

In [2]:
Y = 10
vocab_min = 3

In [3]:
reload(log_reg)
notefile = '../mimicdata/notes_' + str(Y) + '_train_ind.csv'
X_tr = log_reg.construct_csr_matrix(notefile)

Processing...............................................................

In [4]:
X_tr.shape

(15492, 40000)

In [6]:
reload(log_reg)
labelfile = '../mimicdata/labels_10_train.csv'
yy_tr = log_reg.construct_label_lists(labelfile, Y, X_tr.shape[0])

In [7]:
yy_tr

array([[1, 1, 0, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ..., 
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 1, 0, 0],
       [0, 0, 1, ..., 0, 0, 0]])

In [8]:
devfile = '../mimicdata/notes_' + str(Y) + '_dev_ind.csv'
X_dv = log_reg.construct_csr_matrix(devfile)

Processing.................................

In [9]:
X_dv.shape

(7746, 40000)

In [10]:
import evaluation

Still have to de-dupe label set for some strange reason. Maybe I'll just re-export from SQL after doing a DISTINCT

^^ actually, my construct_label_lists takes care of this

In [11]:
labelfile_dev = '../mimicdata/labels_10_dev.csv'
yy_dev = log_reg.construct_label_lists(labelfile_dev, Y, X_dv.shape[0])

In [14]:
from sklearn.metrics import confusion_matrix

In [15]:
#optimize hyperparams of LR
Cs = [1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4]
best_reg = classifiers[0]
best_acc = -np.inf
for C in Cs:
    s = "C = " + str(C) + ": "
    print(s,end="")
    reg = LogisticRegression(C=C,solver='sag')
    reg.fit(X, yy[0])
    yhat = reg.predict(X_dv)
    acc = (yhat == yy_dev[0]).mean()
    print(acc)
    if acc > best_acc:
        best_acc = acc
        best_reg = reg
classifiers[0] = best_reg

C = 0.01: 

C:\Program Files\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:267: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


0.713400464756
C = 0.1: 0.713916860315
C = 1.0: 0.713658662536
C = 10.0: 0.713916860315
C = 100.0: 0.713658662536
C = 1000.0: 0.713916860315
C = 10000.0: 0.713529563646


In [17]:
iters = [1e2, 3e2, 6e2, 1e3, 3e3, 6e3, 1e4]
best_reg = classifiers[0]
best_acc = -np.inf
for max_iter in iters:
    s = "max_iter = " + str(max_iter) + ": "
    print(s,end="")
    reg = LogisticRegression(C=10.0, max_iter=max_iter, solver='sag')
    reg.fit(X, yy[0])
    yhat = reg.predict(X_dv)
    acc = (yhat == yy_dev[0]).mean()
    print(acc)
    if acc > best_acc:
        best_acc = acc
        best_reg = reg
classifiers[0] = best_reg

max_iter = 100.0: 

C:\Program Files\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:267: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


0.713787761425
max_iter = 300.0: 0.691453653499
max_iter = 600.0: 0.678801962303
max_iter = 1000.0: 

KeyboardInterrupt: 

Should split data to balance labels

In [18]:
pcts = sorted([np.array(yy_dev[i]).mean() for i in range(Y)])
pcts
#these are the % of instances containing this label for each of the top 10 labels

[0.18409501678285567,
 0.18874257681383941,
 0.2101729925122644,
 0.24141492383165505,
 0.26452362509682414,
 0.2747224373870385,
 0.3187451587916344,
 0.31900335657113349,
 0.32726568551510454,
 0.55499612703330747]

In [19]:
yhat_tr = classifiers[0].predict(X_tr)

In [20]:
accs = [(yhat_tr == yy[i]).mean() for i in range(Y)]
accs

[0.77956364575264647,
 0.65085205267234703,
 0.45339530080041313,
 0.63665117479989675,
 0.77285050348566997,
 0.88187451587916343,
 0.81493674154402274,
 0.67867286341337463,
 0.86586625355021951,
 0.78563129357087524]

In [22]:
1 - np.array(pcts).mean()
np.array(accs).mean()

0.73202943454686298

Try some other models? SVM, NB

In [23]:
from sklearn.svm import SVC

In [ ]:
clf = SVC(max_iter=100)
clf.fit(X_tr, yy_tr[0])

Try sklearn's multi-label functionality

In [ ]:
yy_tr

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
lr_clf = OneVsRestClassifier(LogisticRegression(solver='sag'))
lr_clf.fit(X_tr, yy_tr_mat)

C:\Program Files\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:267: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Program Files\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:267: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Program Files\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:267: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Program Files\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:267: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Program Files\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:267: ConvergenceWarning: The max_iter was reached which means the coef_

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='sag', tol=0.0001,
          verbose=0, warm_start=False),
          n_jobs=1)

In [ ]:
yhat = lr_clf.predict(X_tr)

In [13]:
yy_tr_mat.shape

(15492, 10)

In [15]:
comp = (yhat == yy_tr_mat)
num_right_per_inst = comp.sum(axis=1)

In [18]:
acc = len([i for i in num_right_per_inst if i == 10])/float(len(num_right_per_inst))
acc

0.08255873999483604